In [51]:
import polars as pl

In [52]:
df = pl.read_parquet("data/*.parquet")
df.head()

created_utc,title,score,upvotes,num_comments,upvote_ratio,is_created_from_ads_ui,total_awards_received,num_reports,image_url,is_video,sub_reddit,created_at
i64,str,i64,i64,i64,f64,bool,i64,null,str,bool,str,datetime[μs]
1767091725,"""Why is the package soft?""",2945,2945,69,0.96,false,0,null,"""https://i.redd.it/uc1h9c7ymbag1.jpeg""",false,"""dankmemes""",2025-12-30 10:48:45
1767105036,"""Well keep hoping""",277,277,3,0.98,false,0,null,"""https://i.redd.it/im9ps31jqcag1.jpeg""",false,"""dankmemes""",2025-12-30 14:30:36
1767044383,"""Saddening tbh""",8674,8674,129,0.95,false,0,null,"""https://i.redd.it/cryqiqb6q7ag1.jpeg""",false,"""dankmemes""",2025-12-29 21:39:43
1767055777,"""I finally solved √(x-3) = 8""",3102,3102,15,0.99,false,0,null,"""https://i.redd.it/knti9uyzn8ag1.jpeg""",false,"""dankmemes""",2025-12-30 00:49:37
1767058788,"""A whole new world""",2175,2175,145,0.86,false,0,null,"""https://i.redd.it/07yus4oyw8ag1.png""",false,"""dankmemes""",2025-12-30 01:39:48


 Since our model will probably use gradient descent lets normalize. And lets make an engament score variable

In [53]:

import time
now = time.time()

df = (
    df.with_columns(
        ((now - pl.col("created_utc")) / 3600)
        .clip(lower_bound=0)
        .alias("age_hours")
    )
    .with_columns(
        (pl.col("upvotes") + 2.0 * pl.col("num_comments"))
        .alias("engagement")
    )
    .with_columns(
        # +1 To avoid function collapse
        (pl.col("engagement") / (pl.col("age_hours")+ 1).log())
        .alias("engagement_score")
    )
)


In [54]:
# Sanity check lets see if these memes are firew
with pl.Config(fmt_str_lengths=50):
    print(df.sort(by="engagement_score",descending=True)["image_url"][0:5])

shape: (5,)
Series: 'image_url' [str]
[
	"https://i.redd.it/y96b05moh9ag1.jpeg"
	"https://i.redd.it/ol3hwvm6r7ag1.png"
	"https://i.redd.it/5imc5f19ucag1.jpeg"
	"https://i.redd.it/uh6ccy55w5ag1.png"
	"https://i.redd.it/dvnz6wn6ybag1.jpeg"
]
